<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/prep_data/HAFS/download_HAFSa_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## To download [HAFSA](https://www.emc.ncep.noaa.gov/hurricane/HFSA/) atmospheric data from [AWS.](https://noaa-nws-hafs-pds.s3.amazonaws.com/index.html)

## Tried to follow [this](https://medium.com/@elliottwobler/how-to-get-started-with-grib2-weather-data-and-python-757df9433d19) notebook on how to read grib2 data file.

In [ ]:
# pynio would be nice - if only pip can find a version!
!pip install cfgrib

!pip install holoviews
!pip install hvplot

## Data URLs

In [ ]:
# 2024/08/22 First available HAFSA MOM6 output
#grb_url = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/20240822/00/07e.2024082200.hfsa.parent.atm.f000.grb2"

#mom_url = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/20240822/00/07e.2024082200.hfsa.mom6.f000.nc"

# 2024/10/06

grb_url = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/20241010/00/14l.2024101000.hfsa.parent.atm.f000.grb2"

mom_url = "https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/20241010/00/14l.2024101000.hfsa.mom6.f000.nc"

grb_idx_url = grb_url + ".idx"

In [ ]:
# atmospheric data
!wget -O hfsa_atm.grb2 {grb_url}
!wget -O hfsa_atm.grb2.idx {grb_idx_url}

# ocean data
!wget -O hfsa_mom.nc {mom_url}

In [ ]:
import xarray as xr
import cfgrib as cfg

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import hvplot.xarray

## Load data
- Atmospheric surface.
- Atmospheric profile (3-d).

In [ ]:
# Filter the dataset to include only data with typeOfLevel 'surface'
ds_atm_sfc = xr.open_dataset("hfsa_atm.grb2", engine="cfgrib",\
                         filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'surface'}, decode_timedelta=True)

In [ ]:
ds_atm_sfc

In [ ]:
(ds_atm_sfc['sp']/100.).hvplot()

In [ ]:
ds_atm = xr.open_dataset("hfsa_atm.grb2", engine="cfgrib",\
                             filter_by_keys={'typeOfLevel': 'isobaricInhPa'}, decode_timedelta=True)

In [ ]:
ds_atm

In [ ]:
atm_mean_prof = ds_atm.sel(longitude=slice(200, 220), latitude=slice(15, 25)).mean(('longitude', 'latitude'))

## Ocean data

In [ ]:
ds_ocn = xr.open_dataset("hfsa_mom.nc", decode_times=False)

ds_ocn = ds_ocn.assign_coords({'geolon': ds_ocn['geolon'],
                       'geolat': ds_ocn['geolat']})

In [ ]:
ds_ocn

In [ ]:
ds_ocn.LwLatSens.isel(time=0).hvplot(x='geolon', y='geolat', kind='quadmesh')

In [ ]:
ocn_mean_prof = ds_ocn.sel(xh=slice(-130, -120), yh=slice(15, 25)).mean(('xh', 'yh'))

In [ ]:
tOffset = 273.15


fig, ax = plt.subplots(2, 1, figsize=(4, 12))

atm_mean_prof.t.plot(ax=ax[0], y='isobaricInhPa', yincrease=False)
(ocn_mean_prof.isel(time=0).temp+tOffset).plot(ax=ax[1], y='z_l', yincrease=False)

ax[0].axis('tight')
ax[1].axis('tight')

ax[0].set_title('')
ax[0].set_xlabel('Atm Temperature [K]')
ax[0].set_ylabel('Pressure (hPa)')

ax[1].set_title('')
ax[1].set_xlabel('Ocean Temperature [$^o$C]')
ax[1].set_ylabel('Depth (m)')

In [ ]:
print(f'Mean of the atmospheric lowest level temperature:\t {atm_mean_prof.t[0].values}\nMean of the ocean highest level temperature:\t {ocn_mean_prof.isel(time=0).temp[0].values+273.15}')